In [3]:
!pip install streamlit==1.39.0
!pip install Pillow

   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.7 MB 31.2 kB/s eta 0:04:40
   ---------------------------------------- 0.0/8.7 MB 31.2 kB/s eta 0:04:40
   ---------------------------------------- 0.0/8.7 MB 31.2 kB/s eta 0:04:40
   ---------------------------------------- 0.0/8.7 MB 31.2 kB/s eta 0:04:40
   ---------------------------------------- 0.0/8.7 MB 31.2 kB/s eta 0:04:40
   ---------------------------------------- 0.0/8.7 MB 31.2 kB/

In [15]:
%%writefile app.py
# Import necessary libraries
import streamlit as st
import numpy as np
import mediapipe as mp
import pickle
import cv2
from PIL import Image, ImageFont, ImageDraw

with open('usl_common_actions.p', 'rb') as f:
    model = pickle.load(f)['model']

# Urdu translations for common phrases
labels_dict = {
    'help': 'میری مدد کرو',
    'i love you': 'میں آپ سے پیار کرتا/کرتی ہوں',
    'no': 'نہیں',
    'please': 'براہ مہربانی',
    'thank you': 'آپ کا شکریہ',
    'yes': 'ہاں',
    'you': 'آپ'
}
# Set up Streamlit page configuration
st.set_page_config(page_title="Urdu Sign Language", page_icon="👋")
st.title("Urdu Sign Language Recognition")
st.write("Press the button below to start recognizing Sign Language")

# Load Mediapipe Hands model
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

# Streamlit video capture function
def process_video():
    cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
    stframe = st.empty()  # Placeholder for video frames
    result_box = st.empty()  # Placeholder for predicted results

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            st.warning("Could not access the webcam.")
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(frame_rgb)

        data_aux = []
        x_ = []
        y_ = []

        if results.multi_hand_landmarks:
            
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                for lm in hand_landmarks.landmark:
                    x_.append(lm.x)
                    y_.append(lm.y)

                for lm in hand_landmarks.landmark:
                    data_aux.append(lm.x - min(x_))
                    data_aux.append(lm.y - min(y_))

                    # Ensure only 42 features (21 landmarks * 2)
                    data_aux = data_aux[:42]

                x1 = int(min(x_) * frame.shape[1]) - 10
                y1 = int(min(y_) * frame.shape[0]) - 10

            
                # Predicting the character
                prediction = model.predict([np.asarray(data_aux)])
                predicted_character = prediction[0]
                predicted_urdu_character = labels_dict.get(predicted_character, "Unknown")

                # Update the result box with the predicted Urdu character
                result_box.text(f"Predicted Urdu Character: {predicted_urdu_character}")

                # Draw bounding box and label
                x1 = int(min(x_) * frame.shape[1]) - 10
                y1 = int(min(y_) * frame.shape[0]) - 10
                x2 = int(max(x_) * frame.shape[1]) + 10
                y2 = int(max(y_) * frame.shape[0]) + 10

                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 4)

                # Create a PIL image from the OpenCV frame
                pil_image = Image.fromarray(frame)
                draw = ImageDraw.Draw(pil_image)

                # Load your Urdu font (make sure to provide the correct path to your font file)
                font_path = './Jameel Noori Nastaleeq Kasheeda.ttf'  # Update this path if necessary
                font = ImageFont.truetype(font_path, 50)

                # Draw the predicted Urdu character
                #draw.text((x1, y1 - 50), predicted_urdu_character, font=font, fill=(0, 0, 0))

                # Convert PIL image back to OpenCV format
                frame = np.array(pil_image)

        # Display the processed video frame
        stframe.image(frame, channels="BGR")

    cap.release()

# Start the video processing function
if st.button("Start"):
    process_video()


Overwriting app.py


In [17]:
!streamlit run app.py

^C
